In [1]:
!pip install requests python-dotenv

In [8]:
import os
import requests
from dotenv import load_dotenv

from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.runnables import RunnablePassthrough

load_dotenv("/content/.env")

True

In [9]:
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=API_KEY,
    api_version="2024-12-01-preview",
    deployment_name="gpt-4.1",
    temperature=0,
)


NameError: name 'AZURE_OPENAI_ENDPOINT' is not defined

In [ ]:
#Define web search
@tool
def web_search(query: str) -> str:
    """Search the web for real-time or current information."""
    url = "https://serpapi.com/search.json"
    params = {
        "q": query,
        "api_key": SERPAPI_KEY,
        "num": 5,
    }

    response = requests.get(url, params=params)
    data = response.json()

    results = data.get("organic_results", [])
    if not results:
        return "No results found."

    return "\n".join(
        r.get("snippet", "") for r in results[:3] if r.get("snippet")
    )


In [ ]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(
        content=(
            "You are an intelligent assistant.\n"
            "If the question requires real-time or current information, "
            "call the appropriate tool.\n"
            "Otherwise answer directly."
        )
    ),
    ("human", "{input}")
])


In [ ]:
#Bind tools to the model
llm_with_tools = llm.bind_tools([web_search])

In [ ]:
#Build runnable chain
chain = (
    {"input": RunnablePassthrough()}
    | prompt
    | llm_with_tools
)


In [ ]:
#Real time tool call
response = chain.invoke("What is the position of the moon relative to Germany right now?")
response

In [ ]:
#To execute tool calls automatically
from langchain_core.messages import ToolMessage

def run_with_tools(chain, user_input):
    result = chain.invoke(user_input)

    if result.tool_calls:
        tool_outputs = []
        for call in result.tool_calls:
            tool_result = web_search.invoke(call["args"])
            tool_outputs.append(
                ToolMessage(
                    content=tool_result,
                    tool_call_id=call["id"]
                )
            )

        final = llm.invoke([result, *tool_outputs])
        return final.content

    return result.content


In [ ]:
#running tools
run_with_tools(
    chain,
    "What is the position of the moon relative to Germany right now?"
)

In [ ]:
run_with_tools(
    chain,
    "Explain quantum entanglement simply."
)